# 🔍 Gadget Spec Scout - Multi-Agent Smartphone Comparison System

**A production-ready ADK implementation for intelligent smartphone comparison**

## 🎯 What This System Does

This multi-agent system helps users make informed smartphone purchase decisions by:

- **📊 Comparing Specifications**: Detailed technical comparisons across devices
- **💰 Finding Best Prices**: Multi-retailer price comparison for Indian market
- **⭐ Analyzing Reviews**: Aggregated user ratings, pros, and cons
- **🤖 Smart Recommendations**: AI-powered suggestions based on user needs

## 🏗️ Architecture

```
┌─────────────────────────────────────────┐
│          Leader Agent                    │
│  (Orchestrates & Recommends)            │
└──────────────┬──────────────────────────┘
               │
     ┌─────────┴──────────┬──────────┐
     │                    │          │
     ▼                    ▼          ▼
┌─────────┐          ┌─────────┐  ┌─────────┐
│  Spec   │          │  Price  │  │ Review  │
│  Agent  │          │  Agent  │  │  Agent  │
└─────────┘          └─────────┘  └─────────┘
```

## ✨ Key Features

- ✅ **Multi-agent architecture** with specialized agents
- ✅ **Session management** for conversation history
- ✅ **Structured tool returns** with error handling
- ✅ **Logging & observability** via LoggingPlugin
- ✅ **Evaluation framework** with test cases
- ✅ **Interactive demo** with sample queries

---

## 📦 Section 1: Setup & Installation

### 1.1: Install Dependencies

In [30]:
!pip install -q google-adk pandas tabulate aiosqlite

### 1.2: Mount Google Drive & Copy MCP Files

This will mount your Google Drive and copy the MCP server files to Colab.

In [31]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:

# Copy MCP files from Drive to Colab
import shutil
import os

# Define paths (CHANGE THESE to match your Drive structure)
DRIVE_MCP_FOLDER = '/content/drive/MyDrive/Kaggle-Google-ADK/gadget-scout-mcp'  # Your MCP folder in Drive
DRIVE_MCP_CLIENT_COLAB = '/content/drive/MyDrive/Kaggle-Google-ADK/mcp_client_colab.py'  # Colab-compatible client

# Copy to Colab's /content/ directory
print("📦 Copying MCP files from Google Drive...")

# Copy MCP server folder
if os.path.exists(DRIVE_MCP_FOLDER):
    if os.path.exists('/content/gadget-scout-mcp'):
        shutil.rmtree('/content/gadget-scout-mcp')
    shutil.copytree(DRIVE_MCP_FOLDER, '/content/gadget-scout-mcp')
    print("✅ Copied gadget-scout-mcp/ folder")
else:
    print(f"❌ MCP folder not found at: {DRIVE_MCP_FOLDER}")
    print("   Please update DRIVE_MCP_FOLDER path above")

# Copy mcp_client_colab.py (Colab-compatible version)
if os.path.exists(DRIVE_MCP_CLIENT_COLAB):
    shutil.copy(DRIVE_MCP_CLIENT_COLAB, '/content/mcp_client_colab.py')
    print("✅ Copied mcp_client_colab.py")
else:
    print(f"❌ mcp_client_colab.py not found at: {DRIVE_MCP_CLIENT_COLAB}")
    print("   Please upload mcp_client_colab.py to your Drive folder")

# Verify files
print("\n🔍 Verifying files...")
if os.path.exists('/content/mcp_client_colab.py'):
    print("✅ mcp_client_colab.py ready")
if os.path.exists('/content/gadget-scout-mcp/server.py'):
    print("✅ MCP server ready")

print("\n✅ All MCP files copied from Google Drive!")

📦 Copying MCP files from Google Drive...
✅ Copied gadget-scout-mcp/ folder
✅ Copied mcp_client.py

🔍 Verifying files...
✅ mcp_client.py ready
✅ MCP server ready

✅ All MCP files copied from Google Drive!


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1.2: Configure API Key

**Using Google Colab Secrets (Recommended):**

1. Click the 🔑 key icon in the left sidebar
2. Add a secret named `GEMINI_API_KEY`
3. Paste your API key (get one at [aistudio.google.com](https://aistudio.google.com/app/apikey))
4. Enable notebook access

**Alternative:** Directly set the API key below (⚠️ Don't share notebooks with embedded keys!)

In [34]:
import os

# Try to get from Colab secrets first
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
    print("✅ API key loaded from Colab secrets")
except:
    # Fallback: direct input
    GEMINI_API_KEY = input("Enter your Gemini API key: ")
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
    print("✅ API key set")

✅ API key loaded from Colab secrets


### 1.3: Import ADK Components

In [35]:
import json
import logging
from typing import List, Dict, Any
from datetime import datetime

# ADK Core
from google.adk.agents import LlmAgent, ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from google.adk.tools import AgentTool
from google.genai.types import Tool, GoogleSearchRetrieval
from google.adk.plugins.logging_plugin import LoggingPlugin

# Google GenAI
from google.genai import types

# Data handling
import pandas as pd
from tabulate import tabulate

print("✅ ADK components imported successfully")

✅ ADK components imported successfully


### 1.4: Configure Retry Options

In [36]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Retry configuration set")

✅ Retry configuration set


---

## 📱 Section 2: Smartphone Database

Mock database with 15 Indian smartphones including specs, prices from 3 retailers, and aggregated reviews.

In [37]:
# Smartphone Database - Specs, Prices, Reviews
SMARTPHONE_DB = {
    "Samsung Galaxy S24 Ultra": {
        "brand": "Samsung",
        "specs": {
            "processor": "Snapdragon 8 Gen 3",
            "ram": "12GB",
            "storage": "256GB",
            "display": "6.8 inch AMOLED, 120Hz",
            "camera": "200MP main, 50MP telephoto, 12MP ultrawide",
            "battery": "5000mAh",
            "os": "Android 14"
        },
        "prices": {
            "Amazon": 124999,
            "Flipkart": 122999,
            "Croma": 124999
        },
        "reviews": {
            "rating": 4.6,
            "total_reviews": 1250,
            "pros": ["Excellent camera", "S Pen support", "Premium build"],
            "cons": ["Expensive", "Heavy"]
        }
    },
    "iPhone 15 Pro Max": {
        "brand": "Apple",
        "specs": {
            "processor": "A17 Pro",
            "ram": "8GB",
            "storage": "256GB",
            "display": "6.7 inch Super Retina XDR, 120Hz",
            "camera": "48MP main, 12MP telephoto, 12MP ultrawide",
            "battery": "4441mAh",
            "os": "iOS 17"
        },
        "prices": {
            "Amazon": 159900,
            "Flipkart": 158900,
            "Croma": 159900
        },
        "reviews": {
            "rating": 4.8,
            "total_reviews": 2100,
            "pros": ["Powerful performance", "iOS ecosystem", "Great camera"],
            "cons": ["Very expensive", "No USB-C charging brick"]
        }
    },
    "OnePlus 12": {
        "brand": "OnePlus",
        "specs": {
            "processor": "Snapdragon 8 Gen 3",
            "ram": "12GB",
            "storage": "256GB",
            "display": "6.82 inch AMOLED, 120Hz",
            "camera": "50MP main, 64MP telephoto, 48MP ultrawide",
            "battery": "5400mAh",
            "os": "OxygenOS 14 (Android 14)"
        },
        "prices": {
            "Amazon": 64999,
            "Flipkart": 63999,
            "Croma": 64999
        },
        "reviews": {
            "rating": 4.5,
            "total_reviews": 890,
            "pros": ["Fast charging", "Good value", "Clean software"],
            "cons": ["No wireless charging", "Average camera performance"]
        }
    },
    "Google Pixel 8 Pro": {
        "brand": "Google",
        "specs": {
            "processor": "Google Tensor G3",
            "ram": "12GB",
            "storage": "256GB",
            "display": "6.7 inch OLED, 120Hz",
            "camera": "50MP main, 48MP telephoto, 48MP ultrawide",
            "battery": "5050mAh",
            "os": "Android 14"
        },
        "prices": {
            "Amazon": 106999,
            "Flipkart": 105999,
            "Croma": 106999
        },
        "reviews": {
            "rating": 4.7,
            "total_reviews": 670,
            "pros": ["Best-in-class camera", "Clean Android", "7 years updates"],
            "cons": ["Gets warm", "Average battery life"]
        }
    },
    "Xiaomi 14": {
        "brand": "Xiaomi",
        "specs": {
            "processor": "Snapdragon 8 Gen 3",
            "ram": "12GB",
            "storage": "256GB",
            "display": "6.36 inch AMOLED, 120Hz",
            "camera": "50MP main, 50MP telephoto, 50MP ultrawide",
            "battery": "4610mAh",
            "os": "HyperOS (Android 14)"
        },
        "prices": {
            "Amazon": 69999,
            "Flipkart": 68999,
            "Croma": 69999
        },
        "reviews": {
            "rating": 4.4,
            "total_reviews": 520,
            "pros": ["Compact size", "Leica camera", "Good performance"],
            "cons": ["MIUI bloatware", "Average battery"]
        }
    }
}

print(f"✅ Database loaded with {len(SMARTPHONE_DB)} smartphones")

✅ Database loaded with 5 smartphones


---

## 🛠️ Section 3: Tool Functions

Implementing tool functions with proper structure, logging, and error handling.

In [38]:
# Tool Functions with Structured Returns

def search_devices(query: str) -> Dict[str, Any]:
    """
    Search for devices matching a query string.

    Args:
        query: Search query (brand, model, or keywords)

    Returns:
        Dictionary with status and matching device names
    """
    try:
        query_lower = query.lower()
        matches = [
            name for name in SMARTPHONE_DB.keys()
            if query_lower in name.lower()
        ]

        if not matches:
            return {
                "status": "error",
                "error_message": f"No devices found matching '{query}'",
                "available_devices": list(SMARTPHONE_DB.keys())
            }

        return {
            "status": "success",
            "data": {
                "query": query,
                "matches": matches,
                "count": len(matches)
            }
        }
    except Exception as e:
        return {"status": "error", "error_message": str(e)}


def get_specs(device_name: str) -> Dict[str, Any]:
    """
    Get technical specifications for a device.

    Args:
        device_name: Exact device name

    Returns:
        Dictionary with status and device specifications
    """
    try:
        if device_name not in SMARTPHONE_DB:
            return {
                "status": "error",
                "error_message": f"Device '{device_name}' not found"
            }

        device = SMARTPHONE_DB[device_name]
        return {
            "status": "success",
            "data": {
                "device_name": device_name,
                "brand": device["brand"],
                "specs": device["specs"]
            }
        }
    except Exception as e:
        return {"status": "error", "error_message": str(e)}


def get_price(device_name: str) -> Dict[str, Any]:
    """
    Get price information from multiple retailers.

    Args:
        device_name: Exact device name

    Returns:
        Dictionary with status and pricing details
    """
    try:
        if device_name not in SMARTPHONE_DB:
            return {
                "status": "error",
                "error_message": f"Device '{device_name}' not found"
            }

        prices = SMARTPHONE_DB[device_name]["prices"]
        lowest_price = min(prices.values())
        lowest_retailer = min(prices, key=prices.get)

        return {
            "status": "success",
            "data": {
                "device_name": device_name,
                "prices": prices,
                "lowest_price": lowest_price,
                "lowest_retailer": lowest_retailer,
                "currency": "INR"
            }
        }
    except Exception as e:
        return {"status": "error", "error_message": str(e)}


def get_reviews(device_name: str) -> Dict[str, Any]:
    """
    Get aggregated user reviews and ratings.

    Args:
        device_name: Exact device name

    Returns:
        Dictionary with status and review summary
    """
    try:
        if device_name not in SMARTPHONE_DB:
            return {
                "status": "error",
                "error_message": f"Device '{device_name}' not found"
            }

        reviews = SMARTPHONE_DB[device_name]["reviews"]
        return {
            "status": "success",
            "data": {
                "device_name": device_name,
                "rating": reviews["rating"],
                "total_reviews": reviews["total_reviews"],
                "pros": reviews["pros"],
                "cons": reviews["cons"]
            }
        }
    except Exception as e:
        return {"status": "error", "error_message": str(e)}


def compare_specs(device1: str, device2: str) -> Dict[str, Any]:
    """
    Compare specifications between two devices.

    Args:
        device1: First device name
        device2: Second device name

    Returns:
        Dictionary with status and side-by-side comparison
    """
    try:
        if device1 not in SMARTPHONE_DB:
            return {"status": "error", "error_message": f"Device '{device1}' not found"}
        if device2 not in SMARTPHONE_DB:
            return {"status": "error", "error_message": f"Device '{device2}' not found"}

        specs1 = SMARTPHONE_DB[device1]["specs"]
        specs2 = SMARTPHONE_DB[device2]["specs"]

        comparison = {}
        for key in specs1.keys():
            comparison[key] = {
                device1: specs1[key],
                device2: specs2.get(key, "N/A")
            }

        return {
            "status": "success",
            "data": {
                "device1": device1,
                "device2": device2,
                "comparison": comparison
            }
        }
    except Exception as e:
        return {"status": "error", "error_message": str(e)}


print("✅ Tool functions created")

✅ Tool functions created


---

## 🤖 Section 4: Multi-Agent System

Building specialist agents and the orchestrating Leader Agent.

### 4.1: Create Specialist Agents

In [39]:
# Section 4: Smart Conversational Agent

# Create a single intelligent agent with ALL tools
gadget_scout_agent = LlmAgent(
    name="GadgetScout",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
        enable_google_search=True  # Web search for unknown phones
    ),
    instruction="""You are Gadget Scout - a friendly, intelligent smartphone advisor for Indian users.

## Your Capabilities:
You have access to a comprehensive smartphone database AND web search. Use them intelligently:

### Tools Available:
1. **search_devices(query)** - Find phones in database matching user's needs
2. **get_specs(device_name)** - Get detailed specifications
3. **get_price(device_name)** - Get prices from Amazon, Flipkart, Croma
4. **get_reviews(device_name)** - Get user ratings, pros/cons
5. **compare_specs(device1, device2)** - Side-by-side comparison
6. **Web Search** - For phones NOT in database or latest news

## How to Respond:

### When user asks about a SPECIFIC phone:
- Search database first using search_devices
- If found: get_specs, get_price, get_reviews
- If NOT found: use web search for specs, price, availability
- Provide comprehensive info: specs, pricing, user feedback

### When user asks for COMPARISON:
- Use compare_specs for database phones
- Explain differences in specs, price, user ratings
- Give clear recommendation based on their priorities

### When user asks for RECOMMENDATIONS:
- Understand their budget, priorities (camera, battery, performance, etc.)
- Use search_devices to filter by criteria
- Get full details (specs, prices, reviews) for top matches
- Suggest 2-3 best options with reasoning

### When user asks general questions:
- Answer conversationally using your knowledge
- Offer to search for specific details if helpful

## Response Style:
- Be conversational and friendly (like ChatGPT)
- Ask clarifying questions if needed
- Provide structured, easy-to-scan information
- Always mention price sources and ratings
- Give actionable recommendations

Remember: You're a helpful advisor, not just a data retrieval bot!
""",
    tools=[
        search_devices,
        get_specs,
        get_price,
        get_reviews,
        compare_specs
    ]
)

print("✅ Smart Conversational Agent created!")
print("   - Single agent with all tools")
print("   - Web search enabled for unknown phones")
print("   - Conversational and adaptive")

✅ Smart Conversational Agent created!
   - Single agent with all tools
   - Web search enabled for unknown phones
   - Conversational and adaptive


### 4.2: Create Leader Agent

---

## 💾 Section 5: Session Management & Runner

Setting up persistent session storage and runner with logging.

In [40]:
# Application constants
APP_NAME = "GadgetSpecScout"
USER_ID = "demo_user"

print("✅ Application constants defined")

✅ Application constants defined


In [41]:
# Create session service for conversation history
session_service = DatabaseSessionService(
    db_url="sqlite+aiosqlite:///gadget_scout_sessions.db"
)

# Create runner with logging plugin
runner = Runner(
    agent=gadget_scout_agent,
    app_name="GadgetSpecScout",
    session_service=session_service,
    plugins=[LoggingPlugin()]  # Automatic observability
)

print("✅ Session management and runner configured")

✅ Session management and runner configured


### 5.1: Initialize MCP Server (Enhanced Intelligence)\n\nThe MCP server provides intelligent context management for brilliant agent responses.

In [ ]:
# Initialize MCP for intelligent context (Colab-Compatible)
USE_MCP = True  # Set to False to disable MCP

if USE_MCP:
    try:
        # Add MCP folder to path
        import sys
        sys.path.append('/content/')
        sys.path.append('/content/gadget-scout-mcp')
        
        # Import Colab-compatible MCP client
        from mcp_client_colab import initialize_colab_mcp, get_colab_mcp_client
        
        # Initialize MCP client (direct import, no stdio)
        mcp_client = initialize_colab_mcp()
        
        if mcp_client:
            print("✅ MCP components loaded successfully!")
            print("   - Conversation tracking enabled")
            print("   - Context generation active")
            print("   - Agent will have brilliant context awareness")
        else:
            print("⚠️ MCP initialization failed")
            USE_MCP = False
            mcp_client = None
            
    except Exception as e:
        print(f"⚠️ MCP not available: {e}")
        print("   Continuing without MCP (basic mode)")
        USE_MCP = False
        mcp_client = None
else:
    print("ℹ️ MCP disabled - Running in basic mode")
    mcp_client = None

### 5.1: Initialize MCP Server (Optional - For Enhanced Intelligence)\n\nThe MCP server provides intelligent context management. This is **optional** but highly recommended for better agent performance.

In [42]:
USE_MCP = True  # Set to False to disable MCP

if USE_MCP:
    try:
        from mcp_client import initialize_mcp, get_mcp_client

        # Initialize MCP client
        mcp_client = await initialize_mcp("gadget-scout-mcp/server.py")
        print("✅ MCP server connected - Enhanced intelligence enabled!")
        print("   Agent will now have brilliant context awareness")
    except Exception as e:
        print(f"⚠️ MCP server not available: {e}")
        print("   Continuing without MCP (basic mode)")
        USE_MCP = False
        mcp_client = None
else:
    print("ℹ️ MCP disabled - Running in basic mode")
    mcp_client = None

⚠️ MCP server not available: fileno
   Continuing without MCP (basic mode)


---

## 🎮 Section 6: Interactive Demo

Test the system with sample queries!

### Helper Function for Running Queries

In [43]:
# MCP-Enhanced Helper Function (Colab-Compatible)
async def run_query(query_text: str, session_id: str = "demo_session"):
    """Run a query with MCP-enhanced context (if available)"""
    
    print("=" * 80)
    print(f"Query: {query_text}")
    print("=" * 80)
    print()
    
    # Get or create session
    session = None
    try:
        session = await session_service.get_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=session_id
        )
    except:
        pass
    
    if session is None:
        session = await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=session_id
        )
        print(f"✅ Created new session: {session_id}")
    else:
        print(f"✅ Using existing session: {session_id}")
    
    # MCP ENHANCEMENT: Get intelligent context
    enhanced_query = query_text
    if USE_MCP and mcp_client:
        try:
            print("🧠 Getting intelligent context from MCP...")
            context = mcp_client.get_query_context(query_text)
            
            if context:
                # Inject context into query
                enhanced_query = f"""{query_text}

<mcp_context>
{context}
</mcp_context>

Use the above context to provide a brilliant, context-aware response.
If the context suggests a direct answer (e.g., from conversation history),
provide it immediately without asking clarifying questions.
"""
                print("✅ MCP context injected")
            else:
                print("⚠️ No context generated, using basic query")
        except Exception as e:
            print(f"⚠️ MCP context error: {e}")
            print("   Continuing with basic query")
    
    print(f"🤖 Agent is thinking...")
    
    query_content = types.Content(
        role="user",
        parts=[types.Part(text=enhanced_query)]
    )
    
    # Run the agent
    response_text = ""
    tools_called = []
    
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session.id,
        new_message=query_content
    ):
        # Check for tool calls
        if hasattr(event, 'content') and event.content and event.content.parts:
            for part in event.content.parts:
                # Track tool calls
                if hasattr(part, 'function_call') and part.function_call:
                    tool_name = part.function_call.name
                    tools_called.append(tool_name)
                    print(f"🛠️  Agent is calling tool: {tool_name}")
                
                # Get final response
                if hasattr(part, 'text') and part.text:
                    if event.is_final_response():
                        response_text = part.text

        # Check for tool results
        if hasattr(event, 'tool_response') and event.tool_response:
             print(f"📥 Received tool result")

    print("-" * 80)
    print(f"📝 Final Response:")
    print(response_text)
    print("-" * 80)
    print()
    
    # MCP ENHANCEMENT: Track this query for future context
    if USE_MCP and mcp_client:
        try:
            mcp_client.track_query(
                query=query_text,
                tools_called=tools_called,
                result_summary=response_text[:200] if response_text else "",
                session_id=session_id
            )
        except Exception as e:
            pass  # Silent fail for tracking
    
    return response_text

print("✅ MCP-Enhanced helper function created (Colab-compatible)")

✅ MCP-Enhanced helper function created
   - Intelligent context injection when MCP is enabled
   - Falls back to basic mode if MCP unavailable


### 6.1: Sample Query 1 - Device Comparison

In [44]:
await run_query(
    "Compare Samsung Galaxy S24 Ultra and iPhone 15 Pro Max. Which is better for photography?",
    session_id="comparison_demo"
)

Query: Compare Samsung Galaxy S24 Ultra and iPhone 15 Pro Max. Which is better for photography?

✅ Using existing session: comparison_demo
🤖 Agent is thinking... (this may take a few seconds)
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-1fbb8af4-bf5f-4c0f-b5e4-6e4ac83d671a
[logging_plugin]    Session ID: comparison_demo
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: GadgetSpecScout
[logging_plugin]    Root Agent: GadgetScout
[logging_plugin]    User Content: text: 'Compare Samsung Galaxy S24 Ultra and iPhone 15 Pro Max. Which is better for photography?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-1fbb8af4-bf5f-4c0f-b5e4-6e4ac83d671a
[logging_plugin]    Starting Agent: GadgetScout
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: GadgetScout
[logging_plugin]    Invocation ID: e-1fbb8af4-bf5f-4c0f-b5e4-6e4ac83d671a
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-fla

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: GadgetScout
[logging_plugin]    Content: function_call: compare_specs
[logging_plugin]    Token Usage - Input: 880, Output: 32
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 615f70b1-a803-4bbd-81ab-2c468b31ebe9
[logging_plugin]    Author: GadgetScout
[logging_plugin]    Content: function_call: compare_specs
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['compare_specs']
🛠️  Agent is calling tool: compare_specs
    Args: {'device1': 'Samsung Galaxy S24 Ultra', 'device2': 'iPhone 15 Pro Max'}
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: compare_specs
[logging_plugin]    Agent: GadgetScout
[logging_plugin]    Function Call ID: adk-50ced7c9-9ca1-4667-9b01-84636410d6bc
[logging_plugin]    Arguments: {'device1': 'Samsung Galaxy S24 Ultra', 'device2': 'iPhone 15 Pro Max'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: compare_specs
[logging_plugin]   

'That\'s a fantastic question, and both the Samsung Galaxy S24 Ultra and the iPhone 15 Pro Max are powerhouse phones when it comes to photography!\n\nHere\'s a quick look at their camera specifications:\n\n**Samsung Galaxy S24 Ultra:**\n*   **Main Camera:** 200MP\n*   **Telephoto Camera:** 50MP\n*   **Ultrawide Camera:** 12MP\n\n**iPhone 15 Pro Max:**\n*   **Main Camera:** 48MP\n*   **Telephoto Camera:** 12MP\n*   **Ultrawide Camera:** 12MP\n\n**Which is better for photography?**\n\nBased purely on the megapixel count, the **Samsung Galaxy S24 Ultra** boasts significantly higher numbers, especially with its 200MP main sensor and 50MP telephoto lens. This can potentially offer more detail in zoomed-in shots and allow for greater cropping flexibility. Its advanced zoom capabilities (including a 100x Space Zoom) are also a strong point for versatility.\n\nThe **iPhone 15 Pro Max**, while having lower megapixel counts, is known for its exceptional computational photography, natural color r

### 6.2: Sample Query 2 - Budget Search

In [45]:
await run_query(
    "I have a budget of 70000 INR. Which phone gives the best value?",
    session_id="budget_demo"
)

Query: I have a budget of 70000 INR. Which phone gives the best value?

✅ Created new session: budget_demo
🤖 Agent is thinking... (this may take a few seconds)
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-2fbadadf-0e47-44f4-8e2f-b7e06b144574
[logging_plugin]    Session ID: budget_demo
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: GadgetSpecScout
[logging_plugin]    Root Agent: GadgetScout
[logging_plugin]    User Content: text: 'I have a budget of 70000 INR. Which phone gives the best value?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-2fbadadf-0e47-44f4-8e2f-b7e06b144574
[logging_plugin]    Starting Agent: GadgetScout
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: GadgetScout
[logging_plugin]    Invocation ID: e-2fbadadf-0e47-44f4-8e2f-b7e06b144574
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash
[logging_plugin]    Agent: GadgetScout
[logging_plugin]   

'That\'s a great budget to work with! To recommend the "best value" phone for you, I need a little more information. "Value" can mean different things to different people.\n\nCould you tell me what\'s most important to you in a smartphone? For example:\n\n*   **Camera quality:** Are you a photography enthusiast?\n*   **Battery life:** Do you need a phone that lasts all day (or more)?\n*   **Performance:** Are you a heavy gamer or multitasker?\n*   **Display quality:** Do you watch a lot of videos or prefer a vibrant screen?\n*   **Operating system:** Do you prefer Android or iOS?\n*   **Storage:** How much storage do you typically need?\n*   **Brand preference:** Do you have any brands you prefer or want to avoid?\n\nOnce I have a better idea of your priorities, I can suggest some excellent options within your 70,000 INR budget!'

### 6.3: Sample Query 3 - Feature-Specific

In [46]:
await run_query(
    "Show me phones with the best battery life and fast charging",
    session_id="feature_demo"
)

Query: Show me phones with the best battery life and fast charging

✅ Created new session: feature_demo
🤖 Agent is thinking... (this may take a few seconds)
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-949817d8-39de-46d8-a1cd-7e366b4dd356
[logging_plugin]    Session ID: feature_demo
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: GadgetSpecScout
[logging_plugin]    Root Agent: GadgetScout
[logging_plugin]    User Content: text: 'Show me phones with the best battery life and fast charging'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-949817d8-39de-46d8-a1cd-7e366b4dd356
[logging_plugin]    Starting Agent: GadgetScout
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: GadgetScout
[logging_plugin]    Invocation ID: e-949817d8-39de-46d8-a1cd-7e366b4dd356
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash
[logging_plugin]    Agent: GadgetScout
[logging_plugin]    Syste

'I couldn\'t directly find devices using "best battery life" and "fast charging" in my database search. However, I can suggest some popular phones known for these features and then fetch their details for you.\n\nPhones that often come up in discussions about excellent battery life and fast charging include:\n\n*   **OnePlus 12**\n*   **Samsung Galaxy S24 Ultra**\n*   **iPhone 15 Pro Max**\n\nWould you like me to get the detailed specifications, pricing, and reviews for any of these, or perhaps all of them, so you can compare their battery and charging capabilities?'

### 6.4: Interactive Query (Your Turn!)

In [47]:
# Try your own query!
your_query = input("Enter your smartphone query: ")
await run_query(your_query, session_id="interactive_demo")

Enter your smartphone query: i phone 16
Query: i phone 16

✅ Created new session: interactive_demo
🤖 Agent is thinking... (this may take a few seconds)
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-52b615d4-0d5a-4fd4-a895-c8fab9da210c
[logging_plugin]    Session ID: interactive_demo
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: GadgetSpecScout
[logging_plugin]    Root Agent: GadgetScout
[logging_plugin]    User Content: text: 'i phone 16'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-52b615d4-0d5a-4fd4-a895-c8fab9da210c
[logging_plugin]    Starting Agent: GadgetScout
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: GadgetScout
[logging_plugin]    Invocation ID: e-52b615d4-0d5a-4fd4-a895-c8fab9da210c
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash
[logging_plugin]    Agent: GadgetScout
[logging_plugin]    System Instruction: 'You are Gadget Scout - a friendly,

"That's a very interesting choice! However, the iPhone 16 hasn't been released yet, and there aren't any official details about it. Apple usually launches new iPhone models in the fall.\n\nWould you like to know about the latest iPhone models, like the iPhone 15 series, or perhaps explore other smartphones currently available? I can help you with specs, prices, and reviews for existing phones!"

---

## 📊 Section 7: Evaluation Framework

Testing system performance with evaluation cases.

### 7.1: Create Evaluation Test Cases

In [48]:
# Evaluation test cases
eval_cases = [
    {
        "test_id": "spec_comparison",
        "query": "Compare the camera specs of OnePlus 12 and Google Pixel 8 Pro",
        "expected_tools": ["get_specs", "compare_specs"],
        "success_criteria": "Should call SpecAgent and provide detailed camera comparison"
    },
    {
        "test_id": "price_check",
        "query": "What's the lowest price for Xiaomi 14?",
        "expected_tools": ["get_price"],
        "success_criteria": "Should call PriceAgent and identify lowest price retailer"
    },
    {
        "test_id": "review_analysis",
        "query": "What do users say about the Samsung Galaxy S24 Ultra?",
        "expected_tools": ["get_reviews"],
        "success_criteria": "Should call ReviewAgent and summarize pros/cons"
    },
    {
        "test_id": "comprehensive_comparison",
        "query": "Help me choose between OnePlus 12 and Xiaomi 14 for best value",
        "expected_tools": ["get_specs", "get_price", "get_reviews"],
        "success_criteria": "Should call all agents and provide data-driven recommendation"
    }
]

print(f"✅ Created {len(eval_cases)} evaluation test cases")

✅ Created 4 evaluation test cases


### 7.2: Run Evaluation

In [49]:
# Simple evaluation runner
print("\n🧪 Running Evaluation Tests...\n")

for test_case in eval_cases:
    print(f"Test: {test_case['test_id']}")
    print(f"Query: {test_case['query']}")
    print(f"Expected: {test_case['success_criteria']}")

    # Run test
    await run_query(
        test_case['query'],
        session_id=f"eval_{test_case['test_id']}"
    )

    print(f"\n{'='*80}\n")


🧪 Running Evaluation Tests...

Test: spec_comparison
Query: Compare the camera specs of OnePlus 12 and Google Pixel 8 Pro
Expected: Should call SpecAgent and provide detailed camera comparison
Query: Compare the camera specs of OnePlus 12 and Google Pixel 8 Pro

✅ Created new session: eval_spec_comparison
🤖 Agent is thinking... (this may take a few seconds)
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-6879e63c-4e6d-4f47-961a-29c485a878ff
[logging_plugin]    Session ID: eval_spec_comparison
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: GadgetSpecScout
[logging_plugin]    Root Agent: GadgetScout
[logging_plugin]    User Content: text: 'Compare the camera specs of OnePlus 12 and Google Pixel 8 Pro'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-6879e63c-4e6d-4f47-961a-29c485a878ff
[logging_plugin]    Starting Agent: GadgetScout
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: GadgetScout

---

## 🧹 Section 8: Cleanup

Clean up resources when done.

In [50]:
import os

# Clean up session database
if os.path.exists("./gadget_scout_sessions.db"):
    os.remove("./gadget_scout_sessions.db")
    print("✅ Session database cleaned up")
else:
    print("ℹ️ No session database to clean up")

✅ Session database cleaned up


---

## 📚 Summary

### ✅ What We Built

- **Multi-agent system** with specialized agents for specs, prices, and reviews
- **Orchestrating Leader Agent** that synthesizes all information
- **Structured tool functions** with proper error handling
- **Session management** for conversation history
- **Logging & observability** via LoggingPlugin
- **Evaluation framework** with test cases

### 🎯 Key ADK Patterns Demonstrated

1. **AgentTool**: Wrapping specialist agents as tools for the leader
2. **Structured Returns**: `{"status": "success", "data": ...}` pattern
3. **Session Persistence**: DatabaseSessionService for conversation memory
4. **Plugins**: LoggingPlugin for automatic observability
5. **Error Handling**: Graceful degradation with informative errors

### 🚀 Next Steps

- Expand database with more devices
- Add real-time price scraping
- Integrate actual review APIs
- Implement memory consolidation
- Deploy to production (Cloud Run, Agent Engine)

---

## 📖 Resources

- [ADK Documentation](https://google.github.io/adk-docs/)
- [Multi-Agent Patterns](https://google.github.io/adk-docs/agents/workflow-agents/)
- [Session Management](https://google.github.io/adk-docs/sessions/)
- [Evaluation Guide](https://google.github.io/adk-docs/evaluate/)

---

**Built with ❤️ using Google ADK**